# PHF TEST for SOM Energia indexed invoicing
set module path in PYTHONPATH , i.e. /home/erp/src/giscedata_facturacio_indexada_som

# Configuration
  * **tariff_name**: Tarifa i.e 2.0A, 3.0A ...
  * **start_date_str**: Initial invoice date at 01:00:00 (yyyy-mm-dd)
  * **end_date_str**: Final invoice date at 00:00:00 (yyyy-mm-dd)
  * **curve_origin**: 'tpl' or 'profile'

## versions 
Not ESIOS params
  
  * **imu**: Impost Municipal en [%]
  * **d**: Desviaments [€/MWh]
  * **k**: Coeficient / Marge comercial [€/MWh]
  * **omie**: Cuota omie/ree [€/MWh]
  * **fe**: Fons Eficiència [€/MWh]
  * **atr**: Diccionari Peatges per periode {P1: 0, P2: ... , Pn: ...} en [€/kWh]
  * **pc**: Diccionari Pagos Capacidad per periode {P1: 0, P2: ... , Pn: ...} en [€/kWh]

## Profiling
Data to profile by REE coeficients

  * **profile_consume**: Diccionari amb consum a perfilar per períodes {P1: , P2:, ..., Pn:} en [kWh] 


In [1]:
tariff_name = '3.0A'
start_date_str = '2018-04-01'
end_date_str = '2018-05-01'
curve_origin = 'profile'
versions = dict(
    imu = 1.5,
    d = 0.25,
    k = 2.0,
    omie = 0.13341,
    fe = 0.27,
    atr = {'P1': 0.018762, 'P2': 0.012575, 'P3': 0.004670, 'P4': 0.0018762, 'P5': 0.012575, 'P6': 0.004670},
    pc = {'P1': 0.008374, 'P2': 0.004304, 'P3': 0.000058, 'P4': 0.008374, 'P5': 0.004304, 'P6': 0.000058},
)
profile_consume = {'P1': 1000, 'P2': 1000, 'P3': 1000, 'P4': 1000, 'P5': 1000, 'P6': 1000}

In [2]:
from tarifes import *
import copy
from datetime import datetime, date
from enerdata.datetime.holidays import get_holidays
from enerdata.contracts.tariff import *
from enerdata.profiles.profile import *
from enerdata.metering.measure import *
from profile import REEInitalProfile
import csv
import copy
from calendar import monthrange

In [3]:
from libfacturacioatr.pool.component import *

In [4]:
esios = '67c6aff80ca331eec78e1f62b7ffc6799e2674d82d57c04104a612db43496db3'
# calc Holidays
start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
HOLIDAYS = []
for d in [start_date, end_date]:
    HOLIDAYS.extend(get_holidays(start_date.year))
HOLIDAYS = set(HOLIDAYS)
HOLIDAYS

{datetime.date(2018, 1, 1),
 datetime.date(2018, 5, 1),
 datetime.date(2018, 8, 15),
 datetime.date(2018, 10, 12),
 datetime.date(2018, 11, 1),
 datetime.date(2018, 12, 6),
 datetime.date(2018, 12, 8),
 datetime.date(2018, 12, 25)}

In [5]:
curve_file = '/home/afita/tmp/edm/02038_3412500.curva'
curve_factor = 1000.0 # W: 1 kW: 1000
print start_date
print end_date
versions

2018-04-01 00:00:00
2018-05-01 00:00:00


{'atr': {'P1': 0.018762,
  'P2': 0.012575,
  'P3': 0.00467,
  'P4': 0.0018762,
  'P5': 0.012575,
  'P6': 0.00467},
 'd': 0.25,
 'fe': 0.27,
 'imu': 1.5,
 'k': 2.0,
 'omie': 0.13341,
 'pc': {'P1': 0.008374,
  'P2': 0.004304,
  'P3': 5.8e-05,
  'P4': 0.008374,
  'P5': 0.004304,
  'P6': 5.8e-05}}

In [6]:
## Profiling code

TARIFFS = {
    '2.0A': T20A,
    '2.0DHA': T20DHA,
    '2.0DHS': T20DHS,
    '2.1A': T21A,
    '2.1DHA': T21DHA,
    '2.1DHS': T21DHS,
    '3.0A': T30A,
    '3.1A': T31A
}
def array_padding(array, length, value=0):
    return array + ([value] * (length - len(array)))

def get_cofs(start, end):
    c = Coefficients()

    while start <= end:
        cofs = REEProfile.get(start.year, start.month)

        c.insert_coefs(cofs)
        # Add one month but always day 1 to avoid month skip, ie
        # 2015-11-04 > 2015-11-02
        start += relativedelta(months=1, day=1)

    return c

def get_profiled_curve(start, end, tariff, consumes):
    t = TARIFFS[tariff]()
    
    measures = []
    curve_start = datetime.combine(start_date, datetime.min.time())
    curve_end = datetime.combine(end_date - relativedelta(days=1), datetime.min.time())
    for period, consume in consumes.items():
        # start 
        measures.append(
             EnergyMeasure(
                 start_date.date(),
                 TariffPeriod(period, 'te'), 0 * 1000,
                 consumption=0 * 1000
             )
        )
        # end 
        measures.append(
             EnergyMeasure(
                 curve_end.date(),
                 TariffPeriod(period, 'te'), consume * 1000,
                 consumption=consume * 1000
             )
        )

        coefs = get_cofs(curve_start, curve_end)
        profiler = Profiler(coefs)
        
        consums = {}
        drag = 'period'
        
        for hour, vals in profiler.profile(t, measures, drag_method=drag):
            d = hour.date()
            # Hacky patch to work for both systems REE Style number of hours
            # and Esios style datetimes
            first_hour = getattr(profiler.coefficient, 'first_hour', 1)
            if first_hour == 1:
                if hour.hour == 0:
                    d -= timedelta(days=1)
            consums.setdefault(d, [])
            consums[d].append(vals['aprox'])
        # print consums
        curve = []
        for day,hours in sorted(consums.items()):
            curve.append(array_padding(hours, 25))
            
    return curve

In [7]:
## TPL code 
def get_tpl_curve(filename):
    curve_data = []
    day = []
    with open(filename, 'rb') as tplfile:
        curve = csv.reader(tplfile, delimiter='\t')
        curve.next()
        curve.next()
        hour = 0
        for line in curve:
            day.append(int(line[1]) * curve_factor)
            if hour < 23:
                hour+=1
            else:
                hour=0
                day.append(0)
                curve_data.append(day)
                day = copy.copy([])

        return curve_data
                
#curve_data = []
# fixe
#curve_data = [[0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 3000, 3000, 2000, 3000, 1000, 0, 0], [0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 3000, 0, 1000, 0], [0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 2000, 3000, 3000, 2000, 0, 0, 0], [0, 1000, 0, 0, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 0, 1000, 0, 1000, 0, 0], [1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 0, 0, 0, 1000, 1000, 0, 0], [0, 1000, 0, 1000, 0, 1000, 0, 1000, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 3000, 3000, 2000, 3000, 0, 0, 0], [0, 1000, 0, 0, 1000, 0, 1000, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 2000, 3000, 3000, 2000, 1000, 0, 0], [0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 3000, 0, 1000, 0], [1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 3000, 3000, 2000, 3000, 0, 0, 0], [1000, 0, 1000, 0, 1000, 0, 1000, 1000, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 2000, 3000, 3000, 3000, 0, 0, 0], [0, 0, 1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 0, 0, 0, 1000, 0, 0], [1000, 0, 0, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 1000, 0, 0, 0, 0, 0, 0], [0, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 2000, 3000, 3000, 2000, 0, 0, 0], [1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 3000, 1000, 0, 0], [1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 2000, 3000, 3000, 2000, 0, 0, 0], [1000, 0, 0, 1000, 0, 1000, 0, 1000, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 3000, 3000, 2000, 2000, 0, 0, 0], [0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 3000, 3000, 3000, 2000, 1000, 0, 0], [0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 0, 1000, 0, 1000, 0, 0], [0, 1000, 0, 0, 1000, 0, 0, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 0, 0, 0, 1000, 1000, 0, 0], [0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 3000, 2000, 3000, 2000, 0, 0, 0], [1000, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 3000, 3000, 2000, 2000, 0, 0, 0], [1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 3000, 1000, 0, 0], [1000, 0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 2000, 3000, 3000, 2000, 0, 0, 0], [0, 1000, 0, 1000, 0, 1000, 0, 1000, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 3000, 2000, 3000, 2000, 0, 0, 0], [1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 0, 0, 1000, 0], [0, 0, 1000, 0, 0, 0, 1000, 0, 0, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 0, 0], [0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 2000, 0, 0, 0], [1000, 0, 1000, 0, 0, 1000, 0, 1000, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 3000, 2000, 3000, 2000, 0, 0, 0]]
#[[0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 3000, 1000, 0, 0],[0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 2000, 3000,3000,2000, 0, 0, 0],[0, 1000, 0, 1000, 0, 0, 1000, 1000, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 1000, 3000, 3000, 2000, 2000, 0, 0, 0],[0, 1000, 0, 0, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000,0,0, 0, 0, 1000, 0, 1000, 0, 0],[1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 0, 0, 0, 1000, 1000, 0, 0],[0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0,0,0, 3000, 3000, 3000, 2000, 1000, 0, 0],[1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 3000, 2000, 3000, 2000, 0, 1000, 0],[1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000,0,0, 1000, 0, 0, 1000, 0, 3000, 3000, 3000, 2000, 0, 0, 0],[1000, 0, 0, 1000, 0, 1000, 0, 1000, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 3000, 3000, 2000, 2000, 0, 0, 0],[0, 1000, 0, 1000, 0, 1000,0,1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 3000, 0, 0, 0],[0, 0, 1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 0, 0, 1000, 0, 0],[0, 1000, 0, 0, 1000,0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 0, 1000, 1000, 0],[0, 1000, 0, 1000, 0, 0, 1000, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 2000, 3000, 3000, 2000, 0, 0, 0],[0, 1000, 0, 0,1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 3000, 3000, 2000, 3000, 0, 0, 0],[1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 2000, 3000, 3000, 2000, 1000, 0, 0],[1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 3000, 3000, 2000, 3000, 0, 0, 0],[1000, 0, 1000, 0, 0, 1000, 0, 1000, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 1000, 2000, 3000,3000, 2000, 0, 0, 0],[0, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 0, 1000, 0],[0, 1000, 0, 0, 0, 1000, 0, 0, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 0, 0,1000, 0, 0, 0],[1000, 0, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 3000, 3000, 2000, 2000, 1000, 0, 0],[0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0,3000, 3000, 3000, 2000, 0, 1000, 0],[0, 1000, 0, 1000, 0, 0, 1000, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 3000, 2000, 3000, 2000, 0, 0, 0],[0, 1000, 0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0,1000, 0, 0, 1000, 3000, 3000, 3000, 2000, 0, 0, 0],[1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 2000, 3000, 3000, 2000, 0, 1000, 0],[1000, 0, 0, 0, 1000, 0, 0, 1000, 0, 0,1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 0, 0, 0, 0],[0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0],[1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0,1000, 0, 0, 1000, 0, 0, 1000, 0, 3000, 3000, 2000, 2000, 0, 0, 0],[1000, 0, 0, 1000, 0, 1000, 0, 1000, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 3000, 3000, 3000, 2000, 0, 0, 0]]
# generada
#for day in range(0, 28):
#    day = copy.copy([1000.0] * 24)
#    day.append(0.0)
#    curve_data.append(day)
     

In [8]:
curve_end = end_date - relativedelta(days=1)
if curve_origin == 'tpl':
    curve_data = get_tpl_curve('/home/afita/tmp/edm/02038_3412500.curva')
if curve_origin == 'profile':
    curve_data = get_profiled_curve(start_date, end_date, tariff_name, profile_consume)    

print len(curve_data)
print len(curve_data[0])
print len(curve_data[-1])
print curve_data
curve = Curve(start_date, curve_origin.upper())
curve.load(curve_data)
consums = {
    'activa': {start_date_str: {'start': start_date,'end': curve_end,'profile': curve_data}},
    'reactiva': {'P1': 1, 'P2': 2, 'P3': 3, 'P4': 4, 'P5': 5, 'P6': 6 }
}
print curve.total_sum

30
25
25
[[15267, 14142, 13586, 13269, 13199, 13319, 13672, 12828, 7268, 7786, 8332, 24586, 25260, 25298, 24640, 8481, 8208, 7990, 7929, 8149, 8845, 10231, 9738, 8987, 0], [5368, 5153, 5021, 4971, 4980, 5125, 5735, 6503, 3640, 4259, 4495, 11230, 11228, 10661, 9439, 3652, 3677, 3701, 3565, 3402, 3311, 3385, 2985, 2671, 0], [6064, 5789, 5648, 5567, 5572, 5755, 6416, 7294, 4096, 4797, 5060, 12682, 12721, 12055, 10643, 4126, 4162, 4192, 4039, 3864, 3763, 3785, 3329, 2975, 0], [6205, 5927, 5782, 5700, 5705, 5889, 6560, 7450, 4183, 4897, 5165, 12946, 12986, 12307, 10868, 4214, 4252, 4282, 4125, 3946, 3840, 3863, 3398, 3037, 0], [6070, 5797, 5653, 5574, 5579, 5758, 6415, 7286, 4089, 4789, 5049, 12651, 12687, 12025, 10622, 4117, 4155, 4184, 4032, 3855, 3754, 3777, 3323, 2971, 0], [6128, 5847, 5705, 5618, 5610, 5785, 6477, 7407, 4215, 4960, 5228, 13045, 13051, 12321, 10751, 4052, 4022, 4025, 3921, 3813, 3822, 3929, 3508, 3167, 0], [15898, 14825, 14273, 13964, 13902, 14091, 14716, 14388, 8881, 1

In [9]:
tarifa = Tarifa30APoolSOM(
    consums, {},
    start_date_str, end_date_str,
    facturacio=1, facturacio_potencia='max',
    data_inici_periode=start_date_str,
    data_final_periode=end_date_str,
    potencies_contractades={'P1': 20, 'P2': 20, 'P3': 20},
    versions={start_date_str: versions},
    holidays=HOLIDAYS,
    esios_token=esios,
    audit=['curve', 'pmd', 'phf', 'pc3_ree', 'perdues'],
)
tarifa.update_dates_consums(
    tarifa.data_inici,
    tarifa.data_final
)

In [10]:
phf = tarifa.phf_calc(curve, start_date)
str(phf)

"['201804010000-201804302300', '20180401000000',\n [['01', [35.800295635183865, 33.04599278631943, 31.697723275968194, 30.911218143024595, 30.708479132497004, 31.003146339570748, 31.823017880524635, 29.87930846460766, 17.037729730894622, 18.27389752766283, 19.55526328423842, 57.55964002870744, 59.08939606567424, 59.031539631321344, 57.431775610085175, 19.77329378834384, 19.12737951567901, 18.688552522529957, 18.55569337479985, 19.088493987949825, 20.741767466311526, 24.07852692217114, 22.94185164199746, 21.12195536970289, 0.0]],\n ['02', [12.533284452605509, 11.999486446697166, 11.654436686710596, 11.535511817108647, 11.569003077053797, 11.909168744566019, 13.360703439714278, 15.153148010826618, 8.542827864666913, 9.994438026864609, 10.545806194483571, 26.4673216070099, 26.471454991772674, 25.11001916532894, 22.192805542862118, 8.528376153982167, 8.58932259097202, 8.649883119845168, 8.337127881472417, 7.979034043741741, 7.792329781307374, 7.982320277265703, 7.031222933881878, 6.2812114

In [11]:
print "curve total: {} PHF total: {}".format(curve.total_sum, phf.total_sum)

curve total: 6000000 PHF total: 14127.1811536


In [12]:
last_month_day = monthrange(start_date.year, start_date.month)[1]
file_prefix = "{0}_{1}".format(
    datetime.strftime(start_date,'%Y%m%d'),
    datetime.strftime(datetime(start_date.year, start_date.month, last_month_day) ,'%Y%m%d'))
prgpncur = Prgpncur('C2_prgpncur_{0}'.format(file_prefix), esios)
prmncur = Prmncur('C2_prmncur_{0}'.format(file_prefix), esios)
perdues = tarifa.perdclass('C2_perd30A_{0}'.format(file_prefix), esios)

In [13]:
dia = 1                              # 1 a fi mes 
hora = 1                             # 0 a 25 (p.e. les 10:00 -> 9)
period_name = 'P3'                   # Període de la la hora seleccionada
IMU = versions['imu'] * 0.01         # en %
PMD = prmncur.get(dia, hora) * 0.001      # en €/MWh
PC3_REE = prgpncur.get(dia, hora) * 0.001     # en €/MWh
PC3 = versions['pc'][period_name]           # en €/kWh
OMIE_REE = versions['omie'] * 0.001  # en €/MWh
PERD = perdues.get(dia, hora) * 0.01      # en %
#K = versions['k'] * 0.001            # en €/MWh
#D = versions['d'] * 0.001            # en €/MWh
K = versions['k']            # en €/kWh
D = versions['d']            # en €/kWh
FONDO_EFI = versions['fe'] * 0.001   # en €/MWh
PA = versions['atr'][period_name]    # en €/kWh

PHF = (1 + IMU) * ((PMD - PC3_REE + PC3 + OMIE_REE) * (1 + PERD) + K + D + FONDO_EFI ) + PA
'{PHF} = (1 + {IMU}) * (({PMD} - {PC3_REE} + {PC3} + {OMIE_REE}) * (1 + {PERD}) + {K} + {D} + {FONDO_EFI}) + {PA}'.format(**locals())

'2.33672696834 = (1 + 0.015) * ((0.0406 - 0.00089 + 5.8e-05 + 0.00013341) * (1 + 0.186) + 2.0 + 0.25 + 0.00027) + 0.00467'

In [14]:
print '{0} €/kWh = {1} € / {2} kWh'.format(phf.get(dia, hora) / (curve.get(dia, hora) / curve_factor), phf.get(dia, hora) ,(curve.get(dia, hora) / curve_factor))  

2.33672696834 €/kWh = 33.0459927863 € / 14.142 kWh


In [15]:
res = tarifa.factura_energia()
res

{'2018-04-01': {'P1': {'consum': [1000.0],
   'cost': [2372.0773265926855],
   'end': ['2018-05-01'],
   'start': ['2018-04-01']},
  'P2': {'consum': [1000.0000000000001],
   'cost': [2362.6822701416045],
   'end': ['2018-05-01'],
   'start': ['2018-04-01']},
  'P3': {'consum': [1000.0000000000001],
   'cost': [2342.3906947182522],
   'end': ['2018-05-01'],
   'start': ['2018-04-01']},
  'P4': {'consum': [1000.0],
   'cost': [2351.151996441597],
   'end': ['2018-05-01'],
   'start': ['2018-04-01']},
  'P5': {'consum': [1000.0000000000001],
   'cost': [2356.8942042144845],
   'end': ['2018-05-01'],
   'start': ['2018-04-01']},
  'P6': {'consum': [999.9999999999999],
   'cost': [2341.9846614774997],
   'end': ['2018-05-01'],
   'start': ['2018-04-01']}}}

In [16]:
from libfacturacioatr.pool.tarifes import aggr_version_comp
aggr_version_comp(res[start_date_str])

{'P1': {'consum': [1000.0, 1000.0],
  'cost': [2372.0773265926855, 2351.151996441597],
  'end': ['2018-05-01', '2018-05-01'],
  'start': ['2018-04-01', '2018-04-01']},
 'P2': {'consum': [1000.0000000000001, 1000.0000000000001],
  'cost': [2362.6822701416045, 2356.8942042144845],
  'end': ['2018-05-01', '2018-05-01'],
  'start': ['2018-04-01', '2018-04-01']},
 'P3': {'consum': [1000.0000000000001, 999.9999999999999],
  'cost': [2342.3906947182522, 2341.9846614774997],
  'end': ['2018-05-01', '2018-05-01'],
  'start': ['2018-04-01', '2018-04-01']}}

In [17]:
print res[start_date_str]['P1']['consum'][0] + res[start_date_str]['P4']['consum'][0]
print res[start_date_str]['P2']['consum'][0] + res[start_date_str]['P5']['consum'][0]
print res[start_date_str]['P3']['consum'][0] + res[start_date_str]['P6']['consum'][0]

2000.0
2000.0
2000.0


In [18]:
print res[start_date_str]['P1']['cost'][0] + res[start_date_str]['P4']['cost'][0]
print res[start_date_str]['P2']['cost'][0] + res[start_date_str]['P5']['cost'][0]
print res[start_date_str]['P3']['cost'][0] + res[start_date_str]['P6']['cost'][0]

4723.22932303
4719.57647436
4684.3753562


In [19]:
print (res[start_date_str]['P1']['cost'][0] + res[start_date_str]['P4']['cost'][0]) / (res[start_date_str]['P1']['consum'][0] + res[start_date_str]['P4']['consum'][0])
print (res[start_date_str]['P2']['cost'][0] + res[start_date_str]['P5']['cost'][0]) / (res[start_date_str]['P2']['consum'][0] + res[start_date_str]['P5']['consum'][0])
print (res[start_date_str]['P3']['cost'][0] + res[start_date_str]['P6']['cost'][0]) / (res[start_date_str]['P3']['consum'][0] + res[start_date_str]['P6']['consum'][0])

2.36161466152
2.35978823718
2.3421876781


In [20]:
tarifa.get_audit_data('pmd')

[('2018-04-01 01', 47.33, 'C2', ''),
 ('2018-04-01 02', 40.6, 'C2', ''),
 ('2018-04-01 03', 37.8, 'C2', ''),
 ('2018-04-01 04', 35.03, 'C2', ''),
 ('2018-04-01 05', 32.64, 'C2', ''),
 ('2018-04-01 06', 33.82, 'C2', ''),
 ('2018-04-01 07', 33.85, 'C2', ''),
 ('2018-04-01 08', 34.77, 'C2', ''),
 ('2018-04-01 09', 34.53, 'C2', ''),
 ('2018-04-01 10', 36.36, 'C2', ''),
 ('2018-04-01 11', 36.39, 'C2', ''),
 ('2018-04-01 12', 36.61, 'C2', ''),
 ('2018-04-01 13', 35.47, 'C2', ''),
 ('2018-04-01 14', 31.82, 'C2', ''),
 ('2018-04-01 15', 29.61, 'C2', ''),
 ('2018-04-01 16', 25.22, 'C2', ''),
 ('2018-04-01 17', 24.35, 'C2', ''),
 ('2018-04-01 18', 31.58, 'C2', ''),
 ('2018-04-01 19', 33.11, 'C2', ''),
 ('2018-04-01 20', 35.16, 'C2', ''),
 ('2018-04-01 21', 37.71, 'C2', ''),
 ('2018-04-01 22', 45.14, 'C2', ''),
 ('2018-04-01 23', 46.49, 'C2', ''),
 ('2018-04-01 24', 39.97, 'C2', ''),
 ('2018-04-02 01', 39.37, 'C2', ''),
 ('2018-04-02 02', 34.48, 'C2', ''),
 ('2018-04-02 03', 28.31, 'C2', ''),
 ('

In [21]:
tarifa.get_audit_data('curve')

[('2018-04-01 01', 15.267, 'PROFILE', ''),
 ('2018-04-01 02', 14.142, 'PROFILE', ''),
 ('2018-04-01 03', 13.586, 'PROFILE', ''),
 ('2018-04-01 04', 13.269, 'PROFILE', ''),
 ('2018-04-01 05', 13.199, 'PROFILE', ''),
 ('2018-04-01 06', 13.319, 'PROFILE', ''),
 ('2018-04-01 07', 13.672, 'PROFILE', ''),
 ('2018-04-01 08', 12.828, 'PROFILE', ''),
 ('2018-04-01 09', 7.268, 'PROFILE', ''),
 ('2018-04-01 10', 7.786, 'PROFILE', ''),
 ('2018-04-01 11', 8.332, 'PROFILE', ''),
 ('2018-04-01 12', 24.586, 'PROFILE', ''),
 ('2018-04-01 13', 25.26, 'PROFILE', ''),
 ('2018-04-01 14', 25.298, 'PROFILE', ''),
 ('2018-04-01 15', 24.64, 'PROFILE', ''),
 ('2018-04-01 16', 8.481, 'PROFILE', ''),
 ('2018-04-01 17', 8.208, 'PROFILE', ''),
 ('2018-04-01 18', 7.99, 'PROFILE', ''),
 ('2018-04-01 19', 7.929, 'PROFILE', ''),
 ('2018-04-01 20', 8.149, 'PROFILE', ''),
 ('2018-04-01 21', 8.845, 'PROFILE', ''),
 ('2018-04-01 22', 10.231, 'PROFILE', ''),
 ('2018-04-01 23', 9.738, 'PROFILE', ''),
 ('2018-04-01 24', 8.987

In [22]:
tarifa.get_audit_data('pc3_ree')

[('2018-04-01 01', 0.91, 'C2', ''),
 ('2018-04-01 02', 0.89, 'C2', ''),
 ('2018-04-01 03', 0.87, 'C2', ''),
 ('2018-04-01 04', 0.85, 'C2', ''),
 ('2018-04-01 05', 0.85, 'C2', ''),
 ('2018-04-01 06', 0.85, 'C2', ''),
 ('2018-04-01 07', 0.85, 'C2', ''),
 ('2018-04-01 08', 0.86, 'C2', ''),
 ('2018-04-01 09', 1.56, 'C2', ''),
 ('2018-04-01 10', 1.59, 'C2', ''),
 ('2018-04-01 11', 1.63, 'C2', ''),
 ('2018-04-01 12', 2.32, 'C2', ''),
 ('2018-04-01 13', 2.33, 'C2', ''),
 ('2018-04-01 14', 2.55, 'C2', ''),
 ('2018-04-01 15', 2.53, 'C2', ''),
 ('2018-04-01 16', 1.83, 'C2', ''),
 ('2018-04-01 17', 1.81, 'C2', ''),
 ('2018-04-01 18', 1.81, 'C2', ''),
 ('2018-04-01 19', 2.05, 'C2', ''),
 ('2018-04-01 20', 2.06, 'C2', ''),
 ('2018-04-01 21', 2.12, 'C2', ''),
 ('2018-04-01 22', 2.16, 'C2', ''),
 ('2018-04-01 23', 2.11, 'C2', ''),
 ('2018-04-01 24', 1.86, 'C2', ''),
 ('2018-04-02 01', 0.89, 'C2', ''),
 ('2018-04-02 02', 0.86, 'C2', ''),
 ('2018-04-02 03', 0.85, 'C2', ''),
 ('2018-04-02 04', 0.84, 'C2

In [23]:
tarifa.get_audit_data('perdues')

[('2018-04-01 01', 18.9, 'C2', ''),
 ('2018-04-01 02', 18.6, 'C2', ''),
 ('2018-04-01 03', 17.9, 'C2', ''),
 ('2018-04-01 04', 17.2, 'C2', ''),
 ('2018-04-01 05', 16.7, 'C2', ''),
 ('2018-04-01 06', 16.0, 'C2', ''),
 ('2018-04-01 07', 15.5, 'C2', ''),
 ('2018-04-01 08', 17.1, 'C2', ''),
 ('2018-04-01 09', 25.4, 'C2', ''),
 ('2018-04-01 10', 26.7, 'C2', ''),
 ('2018-04-01 11', 26.7, 'C2', ''),
 ('2018-04-01 12', 27.2, 'C2', ''),
 ('2018-04-01 13', 26.2, 'C2', ''),
 ('2018-04-01 14', 24.0, 'C2', ''),
 ('2018-04-01 15', 24.4, 'C2', ''),
 ('2018-04-01 16', 23.5, 'C2', ''),
 ('2018-04-01 17', 23.2, 'C2', ''),
 ('2018-04-01 18', 22.1, 'C2', ''),
 ('2018-04-01 19', 21.1, 'C2', ''),
 ('2018-04-01 20', 20.3, 'C2', ''),
 ('2018-04-01 21', 19.2, 'C2', ''),
 ('2018-04-01 22', 18.2, 'C2', ''),
 ('2018-04-01 23', 19.7, 'C2', ''),
 ('2018-04-01 24', 24.3, 'C2', ''),
 ('2018-04-02 01', 17.5, 'C2', ''),
 ('2018-04-02 02', 16.4, 'C2', ''),
 ('2018-04-02 03', 15.6, 'C2', ''),
 ('2018-04-02 04', 15.0, 'C2

In [24]:
tarifa.get_audit_data('phf')

[('2018-04-01 01', 35.800296, '', ''),
 ('2018-04-01 02', 33.045993, '', ''),
 ('2018-04-01 03', 31.697723, '', ''),
 ('2018-04-01 04', 30.911218, '', ''),
 ('2018-04-01 05', 30.708479, '', ''),
 ('2018-04-01 06', 31.003146, '', ''),
 ('2018-04-01 07', 31.823018, '', ''),
 ('2018-04-01 08', 29.879308, '', ''),
 ('2018-04-01 09', 17.03773, '', ''),
 ('2018-04-01 10', 18.273898, '', ''),
 ('2018-04-01 11', 19.555263, '', ''),
 ('2018-04-01 12', 57.55964, '', ''),
 ('2018-04-01 13', 59.089396, '', ''),
 ('2018-04-01 14', 59.03154, '', ''),
 ('2018-04-01 15', 57.431776, '', ''),
 ('2018-04-01 16', 19.773294, '', ''),
 ('2018-04-01 17', 19.12738, '', ''),
 ('2018-04-01 18', 18.688553, '', ''),
 ('2018-04-01 19', 18.555693, '', ''),
 ('2018-04-01 20', 19.088494, '', ''),
 ('2018-04-01 21', 20.741767, '', ''),
 ('2018-04-01 22', 24.078527, '', ''),
 ('2018-04-01 23', 22.941852, '', ''),
 ('2018-04-01 24', 21.121955, '', ''),
 ('2018-04-02 01', 12.533284, '', ''),
 ('2018-04-02 02', 11.999486,

In [25]:
tarifa.dump_audit_data('pmd', '/tmp/PMD.txt')

'/tmp/PMD.txt'

In [26]:
tarifa.dump_audit_data('curve', '/tmp/CURVE.txt')

'/tmp/CURVE.txt'

In [27]:
tarifa.consums['activa']

{'2018-04-01': {'end': datetime.datetime(2018, 4, 30, 0, 0),
  'profile': [[15267,
    14142,
    13586,
    13269,
    13199,
    13319,
    13672,
    12828,
    7268,
    7786,
    8332,
    24586,
    25260,
    25298,
    24640,
    8481,
    8208,
    7990,
    7929,
    8149,
    8845,
    10231,
    9738,
    8987,
    0],
   [5368,
    5153,
    5021,
    4971,
    4980,
    5125,
    5735,
    6503,
    3640,
    4259,
    4495,
    11230,
    11228,
    10661,
    9439,
    3652,
    3677,
    3701,
    3565,
    3402,
    3311,
    3385,
    2985,
    2671,
    0],
   [6064,
    5789,
    5648,
    5567,
    5572,
    5755,
    6416,
    7294,
    4096,
    4797,
    5060,
    12682,
    12721,
    12055,
    10643,
    4126,
    4162,
    4192,
    4039,
    3864,
    3763,
    3785,
    3329,
    2975,
    0],
   [6205,
    5927,
    5782,
    5700,
    5705,
    5889,
    6560,
    7450,
    4183,
    4897,
    5165,
    12946,
    12986,
    12307,
    10868,
    4214,


In [28]:
len(curve_data[0])

25